##### Copyright 2021 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vectorization and XLA compilation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/tf-quant-finance/blob/master/tf_quant_finance/examples/jupyter_notebooks/Vectorization_and_XLA_compilation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/tf-quant-finance/blob/master/tf_quant_finance/examples/jupyter_notebooks/Vectorization_and_XLA_compilation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
#@title Upgrade to TensorFlow 2.5+
!pip install --upgrade tensorflow

In [ ]:
#@title Install TF Quant Finance
!pip install tf-quant-finance


In this notebook we use option pricing as an example problem to illustrate several approaches to optimization:

* Baseline one-at-a-time computation.
* Batched computation.
* Vectorized computation.
* XLA compilation.


If reader is not familiar with the concepts, going through [the training](https://colab.research.google.com/github/google/tf-quant-finance/blob/master/tf_quant_finance/examples/jupyter_notebooks/Introduction_to_TensorFlow_Part_3_-_Advanced_Tensor_Manipulation.ipynb) first is recommended.

The results in this colab were obtained with the public GPU runtime:

`Runtime -> Change runtime type -> Hardware accelerator -> GPU`


In [ ]:
#@title Imports
import numpy as np
import tensorflow as tf
import tf_quant_finance as tff
import time

In [ ]:
# Check that a GPU is available
!nvidia-smi

Thu Jul 22 11:42:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |    224MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Sequential pricing

Consider a simple example of Black-Scholes American option pricing using binomial tree algorithm. Below we generate data for a 1000 random options
and price them all one by one.

In [ ]:
# Generate 1000 random call options
dtype = tf.float64
num_options = 1000
spots = 1 + tf.random.stateless_uniform(
    [num_options], seed=[4, 2], dtype=dtype)
strikes = spots + (0.5 - tf.random.uniform([num_options], dtype=dtype))
# All options expire in in 1 year
expiry = tf.constant(1.0, dtype=dtype)
# Constant discount rates and volatilities
discount_rate = tf.constant(0.01, dtype=dtype)
volatility = tf.constant(1.0, dtype=dtype)
# Randomly assing put/call indicators
is_call_options = tf.cast(tf.random.stateless_binomial(
    [num_options], seed=[4, 2], counts=True, probs=0.5), tf.bool)

Let us first price each option in one-by-one manner

In [ ]:
# Wrap option pricing function with tf.function to perform
# calculation optimizations
option_price = tf.function(tff.black_scholes.option_price_binomial)

In [ ]:
# Pricing with no batching
start_time = time.time()
prices = []
with tf.device('gpu:0'):
  for spot, strike, is_call_option in zip(spots, strikes, is_call_options):
    prices.append(option_price(
        spots=spot, strikes=strike, expiries=expiry,
        discount_rates=discount_rate, 
        volatilities=volatility,
        is_american=True,
        is_call_options=is_call_option,
        dtype=dtype))
print(f"wall time (no batching): {1000 * (time.time() - start_time)} ms")

wall time (no batching): 11126.479625701904 ms


In [ ]:
# Prices of the 1st five options
prices[:5]

[<tf.Tensor: shape=(), dtype=float64, numpy=0.32871543038795475>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.2948546293901379>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.4912944051602193>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.7329574846641592>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.5549488585678107>]

## Batching

We can significantly improve the performance by pricing all the options in a single batch. Batching means that the inputs to the pricing functions are supplied as a single `Tenor` with a batch dimension. In this case `spots`, `strikes` and `is_call_options` are all `Tensor` of shape `[1000]` and can be 
supplied to `option_price` function as is. 

Most of TF ops are vectorized, as are many of the functions in TF Quant Finance, and allow batch calculations. Typically function
documentation includes information whether the function supports batching. If it does, batching of the calculation can significantly reduce run time.


In [ ]:
# We can compute all the prices in a single batch
prices = option_price(
      spots=spots, strikes=strikes, expiries=expiry,
      discount_rates=discount_rate, 
      volatilities=volatility,
      is_american=True,
      is_call_options=is_call_options,
      dtype=dtype)

In [ ]:
%%timeit
with tf.device('gpu:0'):
  option_price(
        spots=spots, strikes=strikes, expiries=expiry,
        discount_rates=discount_rate, 
        volatilities=volatility,
        is_american=True,
        is_call_options=is_call_options,
        dtype=dtype).numpy()

100 loops, best of 5: 10.6 ms per loop


In [ ]:
# Prices of the 1st five options
prices[:5]

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.32871543, 0.29485463, 0.49129441, 0.73295748, 0.55494886])>

## Vectorization

Sometimes batching is not implemented for arguments of a function (that is, some of the arguments are missing the batch shape). In this case, [`tf.vectorized_map`](https://www.tensorflow.org/s/results?q=tf.vectorized_map) might be extremely useful as it can parallelize calculations along the 0-th dimension of the input `Tensor`s. Below we demonstrate how to use vectorized map for the option pricing example.

In [ ]:
# Define a pricing function for the inputs to parallelize on
def pricer_fn(packed_args):
  # Arguments are packed as a tuple
  spot, strike, is_call_option = packed_args
  return option_price(
        spots=spot, strikes=strike, expiries=expiry,
        discount_rates=discount_rate, 
        volatilities=volatility,
        is_american=True,
        is_call_options=is_call_option,
        dtype=dtype)
  
# Wrap the calculation with a tf.function to enable calculation optimization
@tf.function
def vectorized_pricer(spots, strikes, is_call_options):
  # Call pricer_fn on for each element of spots, strikes and is_call_options
  return tf.vectorized_map(pricer_fn, (spots, strikes, is_call_options))

In [ ]:
prices = vectorized_pricer(spots, strikes, is_call_options)

In [ ]:
%%timeit
with tf.device('gpu:0'):
  vectorized_pricer(spots, strikes, is_call_options).numpy()

10 loops, best of 5: 30.3 ms per loop


In [ ]:
# Prices of the 1st five options
prices[:5]

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.32871543, 0.29485463, 0.49129441, 0.73295748, 0.55494886])>

## XLA compilation

Sometimes speed performance of a calculation is unsatisfactory. This happens, for example, when an underlying computation graph consists of a large number of simple calculations and TensorFlow overhead becomes a bottleneck (when running a graph, TensorFlow needs to orchestrate op execution, which can take substatial amount of time).

XLA compiler takes a different approach by generating an LLVM IR targeting the device (CPU/GPU/TPU) from the computational graph.

Refer to the [official XLA page](https://www.tensorflow.org/xla) for more details. XLA Architecture details can be found [here](https://www.tensorflow.org/xla/architecture).

Below we demonstrate how to use XLA Just-in-time (JIT) compiler to price European options under Heston model using Attari approximation.

In [ ]:
# We will price 1000 options

num_options = 1000

# Generate data
dtype = np.float64
mean_reversion = tf.random.uniform(
    shape=[num_options], minval=0.1, maxval=10.0, dtype=dtype) 
thetas = tf.random.uniform(shape=[num_options], minval=0.1, maxval=.5,
                           dtype=dtype) 
variances = tf.random.uniform(shape=[num_options], minval=0.01, maxval=0.5,
                           dtype=dtype) 
discount_factors = tf.random.uniform(
    shape=[num_options], minval=0.8, maxval=0.99, dtype=dtype) 
expiries = tf.random.uniform(
    shape=[num_options], minval=0.1, maxval=10.0, dtype=dtype) 
forwards = 10.0
spots = forwards * discount_factors
volvol = tf.random.uniform(
    shape=[num_options], minval=0.05, maxval=0.9,
    dtype=dtype) 

strikes = tf.random.uniform(
    shape=[num_options], minval=9, maxval=11,dtype=dtype) 

rhos = tf.random.uniform(
    shape=[num_options], minval=-0.8, maxval=0.8, dtype=dtype) 

# Randomly assing put/call indicators
is_call_options = tf.cast(tf.random.stateless_binomial(
    [num_options], seed=[4, 2], counts=True, probs=0.5), tf.bool)


In [ ]:
# Wrap the pricing function with a tf.function
european_option_price = tf.function(
    tff.models.heston.approximations.european_option_price)

Check out the difference betwen GPU and CPU pricing speed (note there are 2vCPUs in a public colab). 
It does not seem that the GPU has done a good job.

In [ ]:
%%timeit -n 10
with tf.device('cpu:0'):
  european_option_price(
        mean_reversion=mean_reversion,
        theta=thetas,
        volvol=volvol,
        rho=rhos,
        variances=variances,
        spots=spots,
        expiries=expiries,
        strikes=strikes,
        discount_factors=discount_factors,
        is_call_options=is_call_options)

10 loops, best of 5: 394 ms per loop


In [ ]:
%%timeit -n 10
with tf.device('gpu:0'):
  european_option_price(
        mean_reversion=mean_reversion,
        theta=thetas,
        volvol=volvol,
        rho=rhos,
        variances=variances,
        spots=spots,
        expiries=expiries,
        strikes=strikes,
        discount_factors=discount_factors,
        is_call_options=is_call_options).numpy()

10 loops, best of 5: 277 ms per loop


Much better performance can be achieved using XLA's just in time compilation. This can be enabled by passing the argument `jit_compile=True` when calling `tf.function`..

In [ ]:
european_option_price_xla = tf.function(
    tff.models.heston.approximations.european_option_price,
    jit_compile=True)

In [ ]:
%%timeit -n 20
with tf.device('gpu:0'):
  european_option_price_xla(
        mean_reversion=mean_reversion,
        theta=thetas,
        volvol=volvol,
        rho=rhos,
        variances=variances,
        spots=spots,
        expiries=expiries,
        strikes=strikes,
        discount_factors=discount_factors,
        is_call_options=is_call_options).numpy()

The slowest run took 14.82 times longer than the fastest. This could mean that an intermediate result is being cached.
20 loops, best of 5: 8.34 ms per loop


This is much better! The calculation now runs much faster. Sometimes we will receive a warning that `The slowest run took x times longer than the fastest`.
This is because on the first run the actual compilation has to be performed (thus, just-in-time compilation). For the successive runs for a new set of inputs, the compiled function is used.


In [ ]:
# Repeat pricing 100k options with 1000 options per batch for
times = []
prices = []
for _ in range(100):
  # Generate new inputs
  dtype = np.float64
  num_options = 1000
  mean_reversion = tf.random.uniform(
      shape=[num_options], minval=0.1, maxval=10.0, dtype=dtype) 
  thetas = tf.random.uniform(shape=[num_options], minval=0.1, maxval=.5,
                            dtype=dtype) 
  variances = tf.random.uniform(shape=[num_options], minval=0.01, maxval=0.5,
                            dtype=dtype) 
  discount_factors = tf.random.uniform(
      shape=[num_options], minval=0.8, maxval=0.99, dtype=dtype) 
  expiries = tf.random.uniform(
      shape=[num_options], minval=0.1, maxval=10.0, dtype=dtype) 
  forwards = 10.0
  spots = forwards * discount_factors
  volvol = tf.random.uniform(
      shape=[num_options], minval=0.05, maxval=0.9,
      dtype=dtype) 

  strikes = tf.random.uniform(
      shape=[num_options], minval=9, maxval=11,dtype=dtype) 

  rhos = tf.random.uniform(
      shape=[num_options], minval=-0.8, maxval=0.8, dtype=dtype) 

  # Randomly assing put/call indicators
  is_call_options = tf.cast(tf.random.stateless_binomial(
      [num_options], seed=[4, 2], counts=True, probs=0.5), tf.bool)

  # Pricing the options for the new inputs using XLA-compiled function
  start_time = time.time()
  with tf.device('gpu:0'):
    prices.append(european_option_price_xla(
        mean_reversion=mean_reversion,
        theta=thetas,
        volvol=volvol,
        rho=rhos,
        variances=variances,
        spots=spots,
        expiries=expiries,
        strikes=strikes,
        discount_factors=discount_factors,
        is_call_options=is_call_options).numpy())
  end_time = time.time()
  times.append(1000 * (end_time - start_time))
print(f"Median time to price a batch of 1k options: {np.median(times)} ms")

Median time to price a batch of 1k options: 8.356451988220215 ms


Note that XLA compilation can be used for CPU devices as well, and can result in significant speed up compared to the non-compiled code. For the option pricing example, we opserve performance improvement of around 15%.

In [ ]:
%%timeit -n 20
with tf.device('cpu:0'):
  european_option_price_xla(
        mean_reversion=mean_reversion,
        theta=thetas,
        volvol=volvol,
        rho=rhos,
        variances=variances,
        spots=spots,
        expiries=expiries,
        strikes=strikes,
        discount_factors=discount_factors,
        is_call_options=is_call_options).numpy()

20 loops, best of 5: 329 ms per loop


**NB** 
* XLA comes with Ahead-of-time (AOT) mode as well. See the [official documentation](https://www.tensorflow.org/xla/tfcompile) for details.
* Not all function can be XLA-compiled. Some TensorFlow ops are not yet supported by XLA. Also, at the moment all shapes must be known at a runtime. Therefore, at the moment XLA compilation fails for the binomial option pricing algorithm above (we change tree size at each iteration of the algorithm).
* XLA-compiled code is not guaranteed to perform better than the non-compiled one.

